In [1]:
from dotenv import load_dotenv
load_dotenv()
import streamlit as st
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.retrievers.self_query.base import SelfQueryRetriever 
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pandas as pd

In [2]:
df = pd.read_csv("courses.csv")
df.head()

,course_id,course_name,credit,description,is_active,department,workload_score,difficulty_level,is_prerequisite,prerequisite
0,521,Information Structures with Python,4.0,This course covers the concepts of the object-...,True,cs,5,3,True,NaN
1,526,Data Structure and Algorithms,4.0,This course covers and relates fundamental com...,True,cs,5,5,True,MET CS 300 and either MET CS 520 or MET CS 521
2,622,Advanced Programming Techniques,4.0,"Polymorphism, containers, libraries, method sp...",True,cs,5,4,False,"MET CS 342, 521, or 526"
3,665,Software Design and Patterns,4.0,"Software design principles, the object-oriente...",True,cs,5,4,False,"MET CS 341 or 342 and MET CS 565, MET CS 526 o..."
4,673,Software Engineering,4.0,Overview of techniques and tools to develop hi...,True,cs,5,3,False,MET CS 342 and at least one 500-level computer...


In [14]:
df2 = df[["course_id", "course_name", "credit"]]
df2["course_id"] = "CS" + df2["course_id"].astype(str)

df2.to_csv("course_short.csv", index=False)
df2.head()

/var/folders/sf/t4mz6hxd5r57wc_lz4mv89v80000gn/T/ipykernel_52049/2864950947.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["course_id"] = "CS" + df2["course_id"].astype(str)


,course_id,course_name,credit
0,CS521,Information Structures with Python,4.0
1,CS526,Data Structure and Algorithms,4.0
2,CS622,Advanced Programming Techniques,4.0
3,CS665,Software Design and Patterns,4.0
4,CS673,Software Engineering,4.0


In [3]:
loader = CSVLoader(file_path="courses.csv", 
                #    metadata_columns=['course_id'], 
                   encoding="utf-8-sig")
documents = loader.load()
vectorstore = Chroma.from_documents(documents, OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [4]:
# metadata_field_info = [
#     AttributeInfo(
#         name="course_id",
#         description="The course id number that has 100, 200, 300, 400, 500, 600, and 700 level",
#         type="string",
#     ),
#     AttributeInfo(
#         name="course_name",
#         description="The name of the course",
#         type="integer",
#     ),
#     AttributeInfo(
#         name="credit",
#         description="The amount of credits form taking the course",
#         type="string",
#     ),
#     AttributeInfo(
#         name="is_active", 
#         description="The availability status of the course in that semester", 
#         type="boolean"
#     ),
#     AttributeInfo(
#         name="department", 
#         description="The department that offers the course", 
#         type="string"
#     ),
#     AttributeInfo(
#         name="workload_score", 
#         description="The amout of time and effort to complete the homework range from 1-5", 
#         type="integer"
#     ),
#     AttributeInfo(
#         name="difficulty_level", 
#         description="The level of diffuculty to complete the course range from 1-5", 
#         type="integer"
#     ),
#     AttributeInfo(
#         name="is_prerequisite", 
#         description="Is the course a prerequisite of other courses", 
#         type="boolean"
#     ),
# ]

# document_content_description = "A detailed description of the course"

# llm = ChatOpenAI(temperature=0, 
#                    model="gpt-3.5-turbo",
#                    max_tokens=None,
#                    timeout=None,
#                    max_retries=2,
#                    )

# self_query_retriever = SelfQueryRetriever.from_llm(
#     llm,
#     vectorstore,
#     document_content_description,
#     metadata_field_info,
#     search_kwargs={'k': 3, 'filter': {'course_id': '521'}}
# )

In [4]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 10, 'filter': {'course_id': '521'}})
retriever.invoke("easy class")

[]

In [7]:
vectorstore.similarity_search("521", filter={'course_id': '521'})

[Document(metadata={'course_id': '521', 'course_name': 'Information Structures with Python', 'credit': '4', 'department': 'cs', 'difficulty_level': '3', 'is_active': 'TRUE', 'is_prerequisite': 'TRUE', 'row': 0, 'source': 'courses.csv', 'workload_score': '5'}, page_content='description: This course covers the concepts of the object-oriented approach to software design and development using Python. It includes a detailed discussion of programming concepts starting with the fundamentals of data types, control structures methods, classes, arrays and strings, and proceeding to advanced topics such as inheritance and polymorphism, creating user interfaces, exceptions and streams. Upon completion of this course students will be able to apply software engineering principles to design and implement Python applications that can be used in with analytics and big data.\nprerequisite: None')]

In [5]:
llm = ChatOpenAI(temperature=0, 
                   model="gpt-3.5-turbo",
                   max_tokens=None,
                   timeout=None,
                   max_retries=2,
                   )

In [6]:
system_prompt = """
You are a student advisor
You will give the class information based on the quiestion that a student ask
Be aware, the students usually call the course as department+course_id such as CS546

"\n\n"
Context:
"{context}"
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

parser = StrOutputParser()

In [7]:
rag_chain =  prompt | llm | parser

# rag_chain = prompt | llm

# for chunk in rag_chain.stream("Is there any Data strucure classes?"):
#     print(chunk, end="", flush=True)

input = "Is there any Data strucure classes?"
context = retriever.invoke(input)

for chunk in rag_chain.stream({"input": input, "context": context}):
    print(chunk, end="", flush=True)

Yes, there are several courses that focus on data structures. In the Computer Science department, you can take courses like CS210 - Data Structures and Algorithms, CS310 - Advanced Data Structures, or CS546 - Data Structures and Analysis of Algorithms. These courses will cover topics such as arrays, linked lists, stacks, queues, trees, graphs, and algorithms for manipulating these data structures. I recommend checking the course catalog for more information on specific offerings and prerequisites.

In [16]:
input = "What did I just ask?"

for chunk in rag_chain.stream({"input": input, "context": context}):
    print(chunk, end="", flush=True)

You just asked for information about a specific course in the Computer Science department.

In [8]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(rag_chain, 
                                                  get_session_history,
                                                  input_messages_key="input"
                                                  )

In [9]:
config = {"configurable": {"session_id": "003"}}
input = "Is there any Python courses?"
context = retriever.invoke(input)

# response = with_message_history.invoke(
#     {"input": input, "context": context},
#     config=config,
# )

for chunk in with_message_history.stream({"input": input, "context": context}, config=config):
    print(chunk, end="", flush=True)


Yes, there are Python courses offered in the Computer Science department. You can enroll in courses like CS101 - Introduction to Python Programming or CS546 - Advanced Python Programming. These courses will cover various aspects of Python programming language and help you develop your skills in Python.

In [10]:
config = {"configurable": {"session_id": "003"}}
input = "What did I just ask?"
context = None

for chunk in with_message_history.stream({"input": input, "context": context}, config=config):
    print(chunk, end="", flush=True)

You asked if there are any Python courses available.

In [32]:
st.set_page_config(
    page_title="Customer response generator", page_icon=":bird:")

st.header("Customer response generator :bird:")
message = st.text_area("customer message")

if message:
    st.write("Generating best practice message...")

    response = with_message_history.invoke(
        {"input": message, "context": context},
        config=config,
    )

    st.info(response)


2024-09-12 14:27:23.881 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-12 14:27:23.881 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-12 14:27:23.955 
  command:

    streamlit run /opt/miniconda3/envs/eagle-llm/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-09-12 14:27:23.957 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-12 14:27:23.958 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-12 14:27:23.959 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-12 14:27:23.959 Thread 'MainThread': missing ScriptRunContext! This warning can be 